In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai
!pip install -qU 'langgraph==0.3.21' 'langchain-google-genai==2.1.2' 'langgraph-prebuilt==0.1.7'

In [3]:
!pip install async-timeout==4.0.0  # Downgrade to fix conflict

In [4]:
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages

class OrderState(TypedDict):
    messages: Annotated[list, add_messages]  # Chat history
    order: list[str]                         # Current order items
    finished: bool                           # Is the order complete?

# System instructions for the chatbot
BARISTABOT_SYSINT = (
    "system",
    "You are BaristaBot, a cafe ordering assistant. Help customers order drinks from the menu."
)

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

In [6]:
def chatbot(state: OrderState) -> OrderState:
    message_history = [BARISTABOT_SYSINT] + state["messages"]
    return {"messages": [llm.invoke(message_history)]}

In [ ]:
!pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai
!pip install -qU 'langgraph==0.3.21' 'langchain-google-genai==2.1.2' 'langgraph-prebuilt==0.1.7'

In [ ]:
from kaggle_secrets import UserSecretsClient
secret_value = UserSecretsClient().get_secret("GOOGLE_API_KEY")  # Use your actual secret name

In [ ]:
from langgraph.graph import StateGraph, END

In [ ]:
graph_builder.add_edge("chatbot", END)  # Fixed: 'END' instead of 'ENO'

In [ ]:
!pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai
!pip install -qU 'langgraph==0.3.21' 'langchain-google-genai==2.1.2' 'langgraph-prebuilt==0.1.7'

In [ ]:
secret_value = UserSecretsClient().get_secret('GOOGLE_API_KEY')

In [ ]:
secret_value = UserSecretsClient().get_secret('GOOGLE_API_KEY')

In [ ]:
from kaggle_secrets import UserSecretsClient

# Fetch the API key (should print without errors)
google_api_key = UserSecretsClient().get_secret("GOOGLE_API_KEY")
print("API key loaded successfully! (First 5 chars:", google_api_key[:5] + "...)")

In [ ]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI

# Set the API key in the environment
os.environ["GOOGLE_API_KEY"] = google_api_key  # From Secrets

# Initialize Gemini
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")  # Try "gemini-1.5-pro" for complex tasks
print("Gemini initialized!")

In [ ]:
from langgraph.graph import StateGraph, END

# Define your state and chatbot logic here
graph_builder = StateGraph(OrderState)
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge("chatbot", END)  # <<< FIXED: 'END' instead of 'ENO'
chat_graph = graph_builder.compile()

In [ ]:
from typing import TypedDict, Annotated, List
from langgraph.graph.message import add_messages

class OrderState(TypedDict):
    messages: Annotated[List[dict], add_messages]  # Stores chat history
    order: List[str]  # Tracks items in the order
    finished: bool  # Flag for order completion

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

def chatbot(state: OrderState) -> OrderState:
    """Generates a response using Gemini."""
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

In [ ]:
from langgraph.graph import StateGraph, START, END  # Add START!

graph_builder = StateGraph(OrderState)
graph_builder.add_node("chatbot", chatbot)

# Set START → chatbot → END
graph_builder.add_edge(START, "chatbot")  # <<< Missing in your code!
graph_builder.add_edge("chatbot", END)

chat_graph = graph_builder.compile()  # Now it will validate

In [ ]:
def human_node(state: OrderState) -> OrderState:
    """Handles user input."""
    last_msg = state["messages"][-1]
    print("AI:", last_msg.content)
    user_input = input("You: ")
    
    if user_input.lower() in {"q", "quit"}:
        state["finished"] = True
    
    return {"messages": [("user", user_input)]}

In [ ]:
graph_builder = StateGraph(OrderState)

# Add nodes
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("human", human_node)

# Define workflow: START → chatbot → human → chatbot → ...
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", "human")

# Loop until user quits
graph_builder.add_conditional_edges(
    "human",
    lambda state: END if state.get("finished", False) else "chatbot"
)

chat_graph = graph_builder.compile()

In [ ]:
from langchain_core.tools import tool

@tool
def get_menu() -> str:
    """Returns the cafe menu."""
    return "MENU:\n- Espresso\n- Latte\n- Cappuccino"

# Bind tools to Gemini
llm_with_tools = llm.bind_tools([get_menu])

In [ ]:
from kaggle_secrets import UserSecretsClient
import os

# Verify the key is loaded correctly
api_key = UserSecretsClient().get_secret("GOOGLE_API_KEY")
print("API key exists:", bool(api_key))  # Should print "True"
os.environ["GOOGLE_API_KEY"] = api_key  # Set it globally

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
try:
    response = llm.invoke("Hello!")
    print("API is working:", response.content)
except Exception as e:
    print("API Error:", str(e))

In [ ]:
state = {"messages": [], "order": [], "finished": False}  # Correct: uses curly braces

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI
from kaggle_secrets import UserSecretsClient
import os

# Load API key
api_key = UserSecretsClient().get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = api_key

# Test connection
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)
try:
    response = llm.invoke("Test message")
    print("✅ API Working:", response.content)
except Exception as e:
    print("❌ API Error:", str(e))
    # If this fails, check:
    # 1. Key validity in Google AI Studio
    # 2. "Internet" enabled in Kaggle settings
    # 3. Gemini API enabled in Google Cloud

✅ API Working: This is a test message.  How can I help you?


In [10]:
from typing import TypedDict, Annotated, List
from langgraph.graph.message import add_messages

class OrderState(TypedDict):
    messages: Annotated[List[dict], add_messages]  # <<< Plural 'messages'
    order: List[str]
    finished: bool

# Initialize properly
state = {
    "messages": [{"role": "system", "content": "You are BaristaBot"}],
    "order": [],
    "finished": False
}

In [11]:
from langgraph.graph import StateGraph, START, END
import time

# Build graph (same as before)
graph_builder = StateGraph(OrderState)
# ... [your existing graph setup code] ...

# Safe execution with timeout
print("BaristaBot: Welcome! Type 'q' to quit.")
state = {"messages": [], "order": [], "finished": False}

max_retries = 3
retry_delay = 2  # seconds

while not state.get("finished"):
    try:
        for attempt in range(max_retries):
            try:
                state = chat_graph.invoke(state)
                break
            except Exception as e:
                if attempt == max_retries - 1:
                    raise
                print(f"Retrying ({attempt + 1}/{max_retries})...")
                time.sleep(retry_delay)
    except KeyboardInterrupt:
        print("\nSession ended by user")
        break
    except Exception as e:
        print(f"Critical error: {str(e)}")
        break

BaristaBot: Welcome! Type 'q' to quit.
Retrying (1/3)...
Retrying (2/3)...
Critical error: Invalid argument provided to Gemini: 400 * GenerateContentRequest.contents: contents is not specified



In [12]:
from typing import List, Dict, Any
import time

# 1. Define proper message structure
def initialize_state() -> Dict[str, Any]:
    return {
        "messages": [
            {"role": "system", "content": "You are BaristaBot, a helpful cafe assistant."},
            {"role": "assistant", "content": "Welcome! What would you like to order today?"}
        ],
        "order": [],
        "finished": False
    }

# 2. Enhanced chat loop with validation
max_retries = 3
retry_delay = 2  # seconds

print("BaristaBot: Welcome! Type 'q' to quit.")
state = initialize_state()

while not state.get("finished"):
    try:
        # Validate messages exist before invoking
        if not state.get("messages"):
            print("Warning: Empty conversation history - resetting")
            state = initialize_state()
            continue
            
        for attempt in range(max_retries):
            try:
                state = chat_graph.invoke(state)
                break
            except Exception as e:
                if "contents is not specified" in str(e):
                    state["messages"].append({
                        "role": "user", 
                        "content": "I didn't catch that. Could you repeat?"
                    })
                if attempt == max_retries - 1:
                    print(f"Final attempt failed: {str(e)}")
                    state["finished"] = True
                    break
                print(f"Retrying ({attempt + 1}/{max_retries})...")
                time.sleep(retry_delay)
                
    except KeyboardInterrupt:
        print("\nSession ended by user")
        break
    except Exception as e:
        print(f"Critical error: {str(e)}")
        break

BaristaBot: Welcome! Type 'q' to quit.
AI: I can help you with:

* **Coffee:** Espresso, Americano, Latte, Cappuccino, Mocha, Macchiato, Flat White, etc.  I can also tell you about our daily specials!
* **Tea:** Black tea, green tea, herbal tea, etc.  We have a wide selection of loose leaf teas.
* **Pastries:** Croissants, muffins, scones, cookies, and more.  Ask about our gluten-free and vegan options.
* **Other:** Sandwiches, salads, juices, smoothies.

Let me know what you're craving, and I'll do my best to assist you!


You:  I did like coffee "q"


AI: I'm sorry, I didn't understand your request. Could you please rephrase or clarify what kind of coffee you'd like?  Perhaps you could tell me the name of the coffee, or describe what you're looking for (e.g., strong, sweet, milky, etc.)?


You:  i did like coffee


AI: Great!  What kind of coffee did you like?  To help me understand your preference better, could you tell me:

* **What type of coffee was it?** (e.g., latte, cappuccino, Americano, etc.)
* **How did you like it prepared?** (e.g., extra shot, extra hot, specific milk type)
* **What did you particularly enjoy about it?** (e.g., the flavor, the strength, the texture)

This will help me suggest something similar for you today!


...

AI: Thank you! I appreciate the positive feedback.  Let me know if you need anything else in the future.  Have a wonderful day!


You:  you too


AI: You too!  Goodbye!


You:  goodbye


AI: Goodbye!

Session ended by user
